In [2]:
import tensorflow as tf
import DataLoader
from tensorflow.keras import layers
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import os
import numpy as np
from config import Config
import matplotlib.pyplot as plt

In [3]:
train_dir = "/Users/nikki/Documents/xyu_iterms/aorta_classification/aortaData/20191108_41对/train"
val_dir = "/Users/nikki/Documents/xyu_iterms/aorta_classification/aortaData/20191108_41对/val"
test_dir = "/Users/nikki/Documents/xyu_iterms/aorta_classification/aortaData/20191108_41对/test"
csv_path = "/Users/nikki/Documents/xyu_iterms/aorta_classification/aortaData/20191108_41对/df.csv"

dataset_train = DataLoader.Dataset()
dataset_test = DataLoader.Dataset()
dataset_val = DataLoader.Dataset()

dataset_train.load_dataset(train_dir)
dataset_test.load_dataset(test_dir)
dataset_val.load_dataset(val_dir)

/Users/nikki/Documents/xyu_iterms/aorta_classification/aortaData/20191108_41对/train
/Users/nikki/Documents/xyu_iterms/aorta_classification/aortaData/20191108_41对/test
/Users/nikki/Documents/xyu_iterms/aorta_classification/aortaData/20191108_41对/val


In [4]:
dataset_val.case_id

['35_event',
 '35_control',
 '34_event',
 '34_control',
 '33_event',
 '33_control',
 '38_control',
 '38_event',
 '36_event',
 '36_control',
 '37_event',
 '37_control',
 '39_control',
 '39_event',
 '41_control',
 '41_event',
 '40_control',
 '40_event']

In [5]:
def handle_case_attributes(csv_path):
    df = pd.read_csv(csv_path,header=0)
    df.drop(["tchol",'hdl','ldl','trig','hypotension'], axis=1, inplace=True)
    df['ane_diam'].fillna(0,inplace=True)
    df.fillna(method='ffill',inplace=True)
    small = LabelBinarizer().fit(df["smallbin"])
    df['smallbin']=small.transform(df['smallbin'])
    dia = LabelBinarizer().fit(df["diabetes"])
    hbp = LabelBinarizer().fit(df["HBP"])
    df['diabetes']=dia.transform(df['diabetes'])
    df['HBP']=hbp.transform(df['HBP'])
    #cs = MinMaxScaler()
    #df[df.columns[2:]] = cs.fit_transform(df[df.columns[2:]])
    #df[df.columns[2:]] = df[df.columns[2:]].astype(np.float32)
    return df

In [6]:
def train_generator():
    for i in dataset_train.case_id:
        img, class_id = dataset_train.load_case_image(i, IMG_ID)
        det = augment.to_deterministic()
        img = det.augment_image(img)
        yield img, class_id

def test_generator():
    for i in dataset_test.case_id:
        img, class_id = dataset_test.load_case_image(i, IMG_ID)[0]
        yield img

In [7]:
df = handle_case_attributes(csv_path)
print(df)

    id  group  sex  age    score2  smallbin        bmi  time_onset  ane_diam  \
0    1      1    0   71 -0.657735         1  22.656250         7.0     5.000   
1    1      0    0   69 -0.260601         1  27.548209        48.0     5.640   
2    2      1    1   33 -2.010208         0  25.380000        12.0     4.700   
3    2      0    1   28 -1.937681         0  29.069767        10.0     0.000   
4    3      0    1   50 -0.880417         0  23.875115       168.0     0.000   
5    3      1    1   49 -0.817507         1  27.379665        24.0     5.100   
6    4      0    1   47 -1.704438         0  40.000000         5.0     0.000   
7    4      1    1   53 -1.705023         1  19.390582        24.0     4.000   
8    5      1    1   50 -0.012975         0  31.141869        24.0     4.980   
9    5      0    1   50 -0.025385         0  26.423570         8.0     5.600   
10   6      0    1   46 -1.950741         0  29.410759         4.0     4.400   
11   6      1    1   50 -1.963092       

In [12]:
pre_model = tf.keras.applications.VGG19(include_top=False, weights='imagenet', 
                                        input_shape=(Config.IMAGE_DIM,Config.IMAGE_DIM,3))
x = pre_model.output
x = layers.GlobalAveragePooling2D()(x)
model = tf.keras.Model(inputs=pre_model.input, outputs=x)
for i in range(14):
    IMG_ID = i+1
    df[str(IMG_ID)]=0
    for j in dataset_train.case_id:
        img = dataset_train.load_case_image(j,IMG_ID)[0]
        #print(img.shape)
        img = np.expand_dims(img,axis=0)
        features = model.predict(img)
        
        features = np.squeeze(features)
        #f = np.mean(features)
        f = np.dot(features,features.T)
        print(f)
        
        cid, group = j.split("_")
        if group=='event' :
            #df[str(IMG_ID)][(df['id']==int(cid))&(df['group']==1)] = f
            df.loc[(df['id']==int(cid))&(df['group']==1), str(IMG_ID)] = f
        else:
            df.loc[(df['id']==int(cid))&(df['group']==0), str(IMG_ID)] = f
            #df[str(IMG_ID)][(df['id']==int(cid))&(df['group']==0)] = f
        
        #print(df)
    
    for j in dataset_val.case_id:
        img = dataset_val.load_case_image(j,IMG_ID)[0]
        #print(img.shape)
        img = np.expand_dims(img,axis=0)
        features = model.predict(img)
        features = np.squeeze(features)
        #f = np.mean(features)
        f = np.dot(features,features.T)
        print(f)
        cid, group = j.split("_")
        if group=='event' :
            #df[str(IMG_ID)][(df['id']==int(cid))&(df['group']==1)] = f
            df.loc[(df['id']==int(cid))&(df['group']==1), str(IMG_ID)] = f
        else:
            df.loc[(df['id']==int(cid))&(df['group']==0), str(IMG_ID)] = f
            #df[str(IMG_ID)][(df['id']==int(cid))&(df['group']==0)] = f
        #print(df)
print(df)
  
    
    
    
    
    

787.7255
862.9851
1230.1107
819.343
1087.2977
570.0541
701.69336
628.97485
622.3067
749.85925
689.5211
726.7069
936.5994
762.5785
792.7317
674.0147
761.66614
622.02124
533.1105
1025.8511
602.9141
1041.8448
730.19824
961.4267
697.7196
813.6659
759.49005
694.12024
882.05237
704.60974
990.76
766.48376
904.9805
563.57385
835.7109
824.75195
925.1897
1072.619
723.62
703.2485
660.7617
932.6425
1130.1699
747.48206
1229.5876
2107.146
799.9073
967.8887
790.83563
1560.2429
873.8374
550.0825
674.3357
805.61993
891.3391
946.3739
866.5358
1500.0485
865.671
586.0764
670.5509
967.54694
1025.6387
958.6667
871.83765
601.90594
1123.396
1284.5771
908.7042
837.37897
632.5162
875.73816
641.46484
638.82
1092.3218
1411.3582
893.15576
1423.2323
678.9231
1135.1378
607.6304
753.3191
877.6154
1927.0913
681.7512
1987.3579
681.22076
814.8993
2078.0332
772.9026
845.5812
641.97205
1063.2904
924.4271
633.0752
920.59863
808.8901
992.6543
691.1599
710.20795
1850.3376
1179.6656
1106.6594
1272.1804
852.26514
716.5105
826.

883.3683
572.1315
1359.6616
1039.9907
896.8491
977.18164
873.91254
902.69543
669.5906
1346.3961
883.53955
1326.9358
597.7136
1346.0071
939.7698
1872.07
910.1621
1594.5007
944.0077
1042.7206
969.2949
545.94617
1364.4487
639.5012
1935.0905
1895.5659
1769.1494
1358.337
1263.4343
786.1299
430.17963
1671.8572
1443.4523
2085.915
1866.4015
1311.2599
823.1432
1963.8677
735.36206
591.6456
773.0463
1805.7332
759.0293
1367.0554
741.80725
700.62036
1307.436
1228.1028
1168.5742
866.0546
1084.5684
1852.877
1284.6687
595.3761
1029.8928
986.17944
771.40375
1366.667
1715.2429
1594.1831
1073.4314
780.1279
1147.3044
830.4275
899.10504
586.2684
878.47943
1506.0798
705.9091
1192.6964
939.66345
1038.5995
907.22473
613.0437
939.03516
1196.3679
744.6745
759.32434
778.4402
963.357
1622.9814
870.84607
847.42737
1369.8649
1060.2388
1070.5895
831.052
1340.8604
1059.0492
664.06946
1189.9009
862.2306
966.0852
671.9799
987.7091
816.54456
1825.3549
1088.623
1960.2917
852.7606
717.2686
925.1191
740.37384
1333.8103
761

In [13]:
df

,id,group,sex,age,score2,smallbin,bmi,time_onset,ane_diam,diabetes,...,5,6,7,8,9,10,11,12,13,14
0,1,1,0,71,-0.657735,1,22.656250,7.0,5.000,1,...,1102.351562,1044.341431,905.165833,767.463501,720.459473,769.864380,681.468262,830.427490,997.134399,1128.177856
1,1,0,0,69,-0.260601,1,27.548209,48.0,5.640,1,...,752.254761,590.086853,605.288696,519.248047,477.994354,481.304565,755.368591,899.105042,887.393555,1024.538452
2,2,1,1,33,-2.010208,0,25.380000,12.0,4.700,0,...,2788.779785,2951.548340,2369.336426,2509.709473,2105.216797,2087.960938,1326.935791,966.085205,1387.928101,1539.543823
3,2,0,1,28,-1.937681,0,29.069767,10.0,0.000,0,...,839.107056,1108.231934,771.404297,918.657898,864.648865,868.516663,883.539551,862.230591,1190.788574,1266.091553
4,3,0,1,50,-0.880417,0,23.875115,168.0,0.000,0,...,1008.090698,761.508301,801.459106,641.298157,1291.407227,1541.469238,873.912537,1340.860352,1154.286377,1298.858276
5,3,1,1,49,-0.817507,1,27.379665,24.0,5.100,0,...,1253.077881,1533.772949,1128.634644,1183.018066,1026.976685,1066.508545,902.695435,1059.049194,1001.314819,1123.367798
6,4,0,1,47,-1.704438,0,40.000000,5.0,0.000,0,...,1329.700806,2134.098633,1805.326904,1607.108154,1470.137085,1671.443970,1359.661621,1369.864868,1595.577881,1609.625488
7,4,1,1,53,-1.705023,1,19.390582,24.0,4.000,0,...,2228.760498,2716.164795,2189.379639,1319.701050,1730.356323,2296.951172,1039.990723,1060.238770,870.531860,1486.687256
8,5,1,1,50,-0.012975,0,31.141869,24.0,4.980,0,...,2777.462402,2432.452881,2282.190918,2183.297607,2143.560303,2117.976562,1872.069946,1825.354858,1700.698975,1871.333862
9,5,0,1,50,-0.025385,0,26.423570,8.0,5.600,0,...,838.391602,811.658813,1164.529053,1349.803467,1296.512939,1219.933594,939.769775,816.544556,960.827393,1038.667358


In [15]:
df.to_csv('/Users/nikki/Documents/xyu_iterms/aorta_classification/aortaData/20191108_41对/df_new.csv', index=False)

In [17]:
import sklearn
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

In [55]:
cs = MinMaxScaler()
df[df.columns[2:]] = cs.fit_transform(df[df.columns[2:]])
df[df.columns[2:]] = df[df.columns[2:]].astype(np.float32)

In [56]:
X = df[df.columns[2:]]
Y = df["group"]

In [88]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

#选择K个最好的特征，返回选择特征后的数据
SelectKBest(chi2, k=2).fit_transform(X, Y)


numpy.ndarray

In [162]:

from sklearn.feature_selection import f_classif
 
from sklearn.feature_selection import SelectKBest
 
# 代表选择特征值的数量
 
selectkBest=SelectKBest(
        f_classif,
        k=5
        )
 
# 选择自变量
 
feature=X
 
# 调用fit方法进行最好特征的选择
 
bestFeature=selectkBest.fit_transform(
        feature,
        Y
        )
 
# 查看关键因子
print(selectkBest.get_support())
feature.columns[selectkBest.get_support()]
 

[False False False False False  True False False False False False False
 False False False False False False False False False False False False
 False False False False  True  True  True  True]


Index(['time_onset', '11', '12', '13', '14'], dtype='object')

In [82]:
from sklearn.feature_selection import VarianceThreshold

#方差选择法，返回值为特征选择后的数据
#参数threshold为方差的阈值
VarianceThreshold(threshold=).fit_transform(X)

ValueError: No feature in X meets the variance threshold 3.00000

In [168]:
for k in range(31):
    selectkBest=SelectKBest(f_classif,k=k+2)
 
    # 选择自变量
 
    feature=X
 
    # 调用fit方法进行最好特征的选择
 
    bestFeature=selectkBest.fit_transform(feature, Y)
 
    # 查看关键因子
    #print(selectkBest.get_support())
    print(feature.columns[selectkBest.get_support()])
    #for i in range(30):
    gbdt=GradientBoostingClassifier(n_estimators=100, max_depth=4)
    score = cross_val_score(gbdt, X[feature.columns[selectkBest.get_support()]], Y, cv=10, scoring='accuracy')
    print(k, score.mean())

Index(['time_onset', '11'], dtype='object')
0 0.5675
Index(['time_onset', '11', '13'], dtype='object')
1 0.51
Index(['time_onset', '11', '12', '13'], dtype='object')
2 0.5375
Index(['time_onset', '11', '12', '13', '14'], dtype='object')
3 0.51
Index(['bmi', 'time_onset', '11', '12', '13', '14'], dtype='object')
4 0.58
Index(['bmi', 'time_onset', 'hyperlipidemia', '11', '12', '13', '14'], dtype='object')
5 0.5425
Index(['bmi', 'time_onset', 'hyperlipidemia', '9', '11', '12', '13', '14'], dtype='object')
6 0.61
Index(['bmi', 'time_onset', 'hs_cad', 'hyperlipidemia', '9', '11', '12', '13',
       '14'],
      dtype='object')
7 0.6325000000000001
Index(['bmi', 'time_onset', 'hs_cad', 'hyperlipidemia', '7', '9', '11', '12',
       '13', '14'],
      dtype='object')
8 0.5700000000000001
Index(['bmi', 'time_onset', 'hs_cad', 'hyperlipidemia', '2', '7', '9', '11',
       '12', '13', '14'],
      dtype='object')
9 0.6075
Index(['bmi', 'time_onset', 'diabetes', 'hs_cad', 'hyperlipidemia', '2', '

In [158]:
gbdt=GradientBoostingClassifier(n_estimators=30, max_depth=4, max_features=5)
score = cross_val_score(gbdt, X[feature.columns[selectkBest.get_support()]], Y, cv=5, scoring='accuracy')
print( score.mean())

0.6125


In [171]:
for k in range(31):
    selectkBest=SelectKBest(f_classif,k=k+2)
 
    # 选择自变量
 
    feature=X
 
    # 调用fit方法进行最好特征的选择
 
    bestFeature=selectkBest.fit_transform(feature, Y)
 
    # 查看关键因子
    #print(selectkBest.get_support())
    #print(feature.columns[selectkBest.get_support()])
    #for i in range(30):
    x = X[feature.columns[selectkBest.get_support()]]
    metric_all = pd.DataFrame()

    from sklearn.linear_model import LogisticRegression
 
    lr = LogisticRegression(C=1000, solver='liblinear', random_state=0)
    metric = cross_val_score(lr, x, Y, cv=10, scoring='accuracy')
    metric.sort()
    metric_all['glm'] = metric[::-1]
 
    # 拟合决策树模型
    from sklearn import tree
 
    tree = tree.DecisionTreeClassifier(criterion='gini')
    metric = cross_val_score(tree, x, Y, cv=10, scoring='accuracy')
    metric.sort()
    metric_all['tree'] = metric[::-1]
 
    # 拟合svm模型
    from sklearn import svm
 
    svc = svm.SVC(C=1.0, kernel='rbf', gamma='auto')
    metric = cross_val_score(svc, x, Y, cv=10, scoring='accuracy')
    metric.sort()
    metric_all['svm'] = metric[::-1]
 
    # 拟合随机森林算法
    from sklearn.ensemble import RandomForestClassifier
 
    RF = RandomForestClassifier(n_estimators=30, criterion='gini', random_state=10)
    metric = cross_val_score(RF, x, Y, cv=10, scoring='accuracy')
    metric.sort()
    metric_all['RandomForest'] = metric[::-1]
 
    # 构造knn最近邻模型
    from sklearn import neighbors
 
    knn = neighbors.KNeighborsClassifier(n_neighbors=10, algorithm='kd_tree')
    metric = cross_val_score(estimator=knn, X=x, y=Y, cv=10, scoring='accuracy')
    metric.sort()
    metric_all['knn'] = metric[::-1]
 
    # 构造lda模型
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
 
    lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage=None, priors=None)
    metric = cross_val_score(estimator=lda, X=x, y=Y, cv=10, scoring='accuracy')
    metric.sort()
    metric_all['lda'] = metric[::-1]
 
    # 构造qda模型
    from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
 
    qda = QuadraticDiscriminantAnalysis()
    metric = cross_val_score(estimator=qda, X=x, y=Y, cv=10, scoring='accuracy')
    metric.sort()
    metric_all['qda'] = metric[::-1]
 
    #  高斯朴素贝叶斯,多项式贝叶斯适用于文本分类，伯努利贝叶斯需要全部变量威二值变量
    from sklearn.naive_bayes import GaussianNB
 
    gnb = GaussianNB()
    metric = cross_val_score(estimator=gnb, X=x, y=Y, cv=10, scoring='accuracy')
    metric.sort()
    metric_all['gnb'] = metric[::-1]
 
    # sklearn提供了bp多层神经网络，隐含层设为 (4, 3, 2)
    from sklearn.neural_network import MLPClassifier
 
    mbp = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(4, 3, 2), random_state=1)
    metric = cross_val_score(estimator=mbp, X=x, y=Y, cv=10, scoring='accuracy')
    metric.sort()
    metric_all['mbp'] = metric[::-1]
 
    # 采用adaboost算法
    from sklearn.ensemble import AdaBoostClassifier
 
    # 弱分类器的参数 base_estimator 默认为决策树
    ada = AdaBoostClassifier(n_estimators=100)
    metric = cross_val_score(estimator=ada, X=x, y=Y, cv=10, scoring='accuracy')
    metric.sort()
    metric_all['ada'] = metric[::-1]
 
    # 将10种模型结果进行比较
    print(metric_all)
    print(k+2,"个特征按照accuracy进行排序")
    metric_mean = metric_all.mean()
    print(metric_mean.sort_values(ascending=False))
    #score = cross_val_score(g, X[feature.columns[selectkBest.get_support()]], Y, cv=10, scoring='accuracy')
    #print(k, score.mean())

     glm   tree    svm  RandomForest    knn    lda    qda    gnb  mbp    ada
0  0.875  0.800  0.750         0.750  0.875  0.875  0.875  0.875  0.5  0.625
1  0.875  0.750  0.750         0.625  0.750  0.875  0.750  0.625  0.5  0.625
2  0.875  0.625  0.625         0.625  0.750  0.750  0.625  0.625  0.5  0.625
3  0.750  0.625  0.625         0.500  0.750  0.750  0.625  0.625  0.5  0.600
4  0.750  0.625  0.625         0.500  0.625  0.750  0.625  0.625  0.5  0.500
5  0.625  0.625  0.625         0.500  0.625  0.625  0.625  0.625  0.5  0.500
6  0.625  0.625  0.500         0.500  0.600  0.625  0.625  0.500  0.5  0.500
7  0.625  0.500  0.500         0.500  0.500  0.625  0.500  0.500  0.5  0.375
8  0.625  0.500  0.500         0.375  0.500  0.500  0.500  0.500  0.5  0.250
9  0.500  0.500  0.500         0.375  0.250  0.500  0.500  0.500  0.5  0.125
2 个特征按照accuracy进行排序
glm             0.7125
lda             0.6875
qda             0.6250
knn             0.6225
tree            0.6175
gnb             0.

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variab

     glm   tree    svm  RandomForest    knn    lda    qda    gnb    mbp    ada
0  0.750  0.750  0.750         0.750  0.875  0.750  0.625  0.875  0.750  0.875
1  0.750  0.750  0.750         0.750  0.750  0.750  0.625  0.750  0.750  0.800
2  0.750  0.750  0.750         0.750  0.750  0.750  0.625  0.625  0.750  0.625
3  0.750  0.625  0.750         0.750  0.750  0.750  0.625  0.500  0.750  0.625
4  0.750  0.625  0.625         0.700  0.750  0.625  0.500  0.500  0.625  0.625
5  0.750  0.625  0.625         0.625  0.625  0.625  0.500  0.500  0.625  0.625
6  0.625  0.625  0.625         0.625  0.625  0.625  0.500  0.500  0.625  0.500
7  0.625  0.500  0.625         0.625  0.625  0.625  0.500  0.500  0.625  0.500
8  0.600  0.500  0.625         0.500  0.500  0.600  0.500  0.500  0.500  0.500
9  0.375  0.250  0.600         0.250  0.500  0.375  0.375  0.500  0.500  0.375
9 个特征按照accuracy进行排序
knn             0.6750
svm             0.6725
glm             0.6725
mbp             0.6500
lda             0.6

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variab

     glm   tree    svm  RandomForest    knn    lda    qda    gnb    mbp    ada
0  0.750  0.875  0.750         0.750  0.750  0.750  0.875  0.875  0.750  0.875
1  0.750  0.750  0.750         0.750  0.750  0.750  0.750  0.750  0.750  0.750
2  0.750  0.750  0.750         0.750  0.750  0.750  0.750  0.625  0.750  0.625
3  0.750  0.700  0.625         0.750  0.625  0.750  0.625  0.500  0.750  0.625
4  0.750  0.625  0.625         0.700  0.625  0.750  0.625  0.500  0.750  0.625
5  0.750  0.625  0.625         0.625  0.500  0.625  0.600  0.500  0.750  0.625
6  0.625  0.625  0.625         0.625  0.500  0.625  0.500  0.500  0.625  0.625
7  0.625  0.500  0.625         0.625  0.500  0.600  0.500  0.500  0.625  0.600
8  0.600  0.500  0.625         0.625  0.500  0.500  0.375  0.500  0.600  0.500
9  0.375  0.250  0.600         0.375  0.500  0.375  0.250  0.500  0.500  0.375
10 个特征按照accuracy进行排序
mbp             0.6850
glm             0.6725
svm             0.6600
RandomForest    0.6575
lda             0.

...

     glm   tree    svm  RandomForest    knn    lda    qda    gnb  mbp    ada
0  0.750  0.750  0.875         0.750  0.750  0.750  0.750  1.000  0.5  0.750
1  0.750  0.750  0.750         0.750  0.750  0.750  0.625  0.750  0.5  0.625
2  0.625  0.625  0.750         0.750  0.625  0.625  0.625  0.625  0.5  0.625
3  0.625  0.625  0.750         0.750  0.625  0.625  0.625  0.625  0.5  0.625
4  0.625  0.625  0.750         0.625  0.625  0.625  0.500  0.500  0.5  0.625
5  0.625  0.600  0.625         0.625  0.625  0.625  0.500  0.500  0.5  0.500
6  0.500  0.500  0.500         0.625  0.625  0.600  0.375  0.500  0.5  0.400
7  0.500  0.500  0.375         0.500  0.500  0.500  0.375  0.500  0.5  0.375
8  0.500  0.500  0.375         0.500  0.500  0.375  0.125  0.500  0.5  0.250
9  0.375  0.500  0.300         0.400  0.400  0.375  0.125  0.500  0.5  0.250
30 个特征按照accuracy进行排序
RandomForest    0.6275
svm             0.6050
knn             0.6025
gnb             0.6000
tree            0.5975
glm             0

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variab

     glm   tree    svm  RandomForest    knn    lda    qda    gnb  mbp    ada
0  0.875  0.750  0.750         0.750  0.750  0.750  0.500  1.000  0.5  0.750
1  0.750  0.625  0.750         0.750  0.750  0.750  0.500  0.750  0.5  0.625
2  0.625  0.625  0.750         0.625  0.625  0.750  0.500  0.625  0.5  0.625
3  0.625  0.625  0.750         0.625  0.625  0.750  0.500  0.625  0.5  0.625
4  0.625  0.625  0.750         0.625  0.625  0.700  0.500  0.500  0.5  0.625
5  0.625  0.500  0.625         0.625  0.625  0.625  0.375  0.500  0.5  0.500
6  0.625  0.500  0.500         0.500  0.625  0.625  0.375  0.500  0.5  0.400
7  0.500  0.500  0.375         0.500  0.500  0.500  0.375  0.500  0.5  0.375
8  0.500  0.500  0.375         0.500  0.400  0.500  0.375  0.500  0.5  0.250
9  0.375  0.400  0.300         0.375  0.375  0.375  0.125  0.500  0.5  0.250
31 个特征按照accuracy进行排序
lda             0.6325
glm             0.6125
gnb             0.6000
svm             0.5925
knn             0.5900
RandomForest    0

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variab

     glm   tree    svm  RandomForest    knn    lda    qda    gnb  mbp    ada
0  0.875  0.750  0.750         0.750  0.750  0.750  0.625  1.000  0.5  0.750
1  0.750  0.750  0.750         0.750  0.750  0.750  0.500  0.750  0.5  0.625
2  0.625  0.625  0.750         0.750  0.750  0.750  0.500  0.625  0.5  0.625
3  0.625  0.625  0.750         0.625  0.750  0.700  0.500  0.625  0.5  0.625
4  0.625  0.625  0.750         0.625  0.625  0.625  0.500  0.500  0.5  0.625
5  0.625  0.625  0.625         0.625  0.625  0.625  0.500  0.500  0.5  0.500
6  0.500  0.500  0.500         0.625  0.625  0.500  0.500  0.500  0.5  0.400
7  0.500  0.500  0.375         0.500  0.625  0.500  0.375  0.500  0.5  0.375
8  0.500  0.500  0.375         0.500  0.375  0.500  0.375  0.500  0.5  0.250
9  0.375  0.400  0.300         0.250  0.300  0.375  0.125  0.500  0.5  0.250
32 个特征按照accuracy进行排序
knn             0.6175
lda             0.6075
gnb             0.6000
RandomForest    0.6000
glm             0.6000
svm             0

In [81]:
metric_all = pd.DataFrame()

from sklearn.linear_model import LogisticRegression
 
lr = LogisticRegression(C=1000, solver='liblinear', random_state=0)
metric = cross_val_score(lr, X, Y, cv=10, scoring='accuracy')
metric.sort()
metric_all['glm'] = metric[::-1]
 
# 拟合决策树模型
from sklearn import tree
 
tree = tree.DecisionTreeClassifier(criterion='gini')
metric = cross_val_score(tree, X, Y, cv=10, scoring='accuracy')
metric.sort()
metric_all['tree'] = metric[::-1]
 
# 拟合svm模型
from sklearn import svm
 
svc = svm.SVC(C=1.0, kernel='rbf', gamma='auto')
metric = cross_val_score(svc, X, Y, cv=10, scoring='accuracy')
metric.sort()
metric_all['svm'] = metric[::-1]
 
# 拟合随机森林算法
from sklearn.ensemble import RandomForestClassifier
 
RF = RandomForestClassifier(n_estimators=30, criterion='gini', random_state=10)
metric = cross_val_score(RF, X, Y, cv=10, scoring='accuracy')
metric.sort()
metric_all['RandomForest'] = metric[::-1]
 
# 构造knn最近邻模型
from sklearn import neighbors
 
knn = neighbors.KNeighborsClassifier(n_neighbors=10, algorithm='kd_tree')
metric = cross_val_score(estimator=knn, X=X, y=Y, cv=10, scoring='accuracy')
metric.sort()
metric_all['knn'] = metric[::-1]
 
# 构造lda模型
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
 
lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage=None, priors=None)
metric = cross_val_score(estimator=lda, X=X, y=Y, cv=10, scoring='accuracy')
metric.sort()
metric_all['lda'] = metric[::-1]
 
# 构造qda模型
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
 
qda = QuadraticDiscriminantAnalysis()
metric = cross_val_score(estimator=qda, X=X, y=Y, cv=10, scoring='accuracy')
metric.sort()
metric_all['qda'] = metric[::-1]
 
#  高斯朴素贝叶斯,多项式贝叶斯适用于文本分类，伯努利贝叶斯需要全部变量威二值变量
from sklearn.naive_bayes import GaussianNB
 
gnb = GaussianNB()
metric = cross_val_score(estimator=gnb, X=X, y=Y, cv=10, scoring='accuracy')
metric.sort()
metric_all['gnb'] = metric[::-1]
 
# sklearn提供了bp多层神经网络，隐含层设为 (4, 3, 2)
from sklearn.neural_network import MLPClassifier
 
mbp = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(4, 3, 2), random_state=1)
metric = cross_val_score(estimator=mbp, X=X, y=Y, cv=10, scoring='accuracy')
metric.sort()
metric_all['mbp'] = metric[::-1]
 
# 采用adaboost算法
from sklearn.ensemble import AdaBoostClassifier
 
# 弱分类器的参数 base_estimator 默认为决策树
ada = AdaBoostClassifier(n_estimators=100)
metric = cross_val_score(estimator=ada, X=X, y=Y, cv=10, scoring='accuracy')
metric.sort()
metric_all['ada'] = metric[::-1]
 
# 将10种模型结果进行比较
print(metric_all)
print("按照accuracy进行排序")
metric_mean = metric_all.mean()
print(metric_mean.sort_values(ascending=False))

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variab

     glm   tree    svm  RandomForest    knn    lda    qda    gnb  mbp    ada
0  0.875  0.750  0.750         0.750  0.750  0.750  0.625  1.000  0.5  0.750
1  0.750  0.750  0.750         0.750  0.750  0.750  0.500  0.750  0.5  0.625
2  0.625  0.750  0.750         0.750  0.750  0.750  0.500  0.625  0.5  0.625
3  0.625  0.625  0.750         0.625  0.750  0.700  0.500  0.625  0.5  0.625
4  0.625  0.625  0.750         0.625  0.625  0.625  0.500  0.500  0.5  0.625
5  0.625  0.500  0.625         0.625  0.625  0.625  0.500  0.500  0.5  0.500
6  0.500  0.500  0.500         0.625  0.625  0.500  0.500  0.500  0.5  0.400
7  0.500  0.500  0.375         0.500  0.625  0.500  0.375  0.500  0.5  0.375
8  0.500  0.500  0.375         0.500  0.375  0.500  0.375  0.500  0.5  0.250
9  0.375  0.500  0.300         0.250  0.300  0.375  0.125  0.500  0.5  0.250
按照accuracy进行排序
knn             0.6175
lda             0.6075
gnb             0.6000
RandomForest    0.6000
tree            0.6000
glm             0.6000
